Problem 1. (45 points)
In this exercise, you are supposed to code DFS and related algorithms done in class. To help you, we are
providing Python code for a graph class file graph_class.py which you can import. You should feel
completely free to modify this code as you please. The operations that this gives you are:

* init: G = graph() would create an empty graph
* Read Edges: G.Read Edges(fname) would read directed edges from the file “fname” where eachline would contains endpoints of one directed edge separated by whitespace.
* Other routines are sufficiently commented to be clear. Ask questions on Ed Discussion for clarification



In [137]:
# import in graph class
from graph_class import graph

a. Implement DFS. Read the directed graph in the file graph2.txt and run DFS on it. When you
implement DFS, you should traverse neighbors (the For-loop of DFS) in alphabetical order1. The
order σ in which you run DFS should also be alphabetical. So in particular, you should start exploring
from the letter a. Write down the order in which the vertices are visited. Do you agree? (10 points)

In [2]:
'''read in graph from graph2.txt'''
# initalize graph
G = graph()
# read in edges
G.Read_Edges("graph2.txt")
G.Read_Vertex_Labels("graph2.txt")

In [3]:
# check to make sure has read in correctly
print(G.num_vertices(),G.num_edges())

13 20


In [97]:
def DFS(G, ordering):
    global vert_to_index; vert_to_index = {}
    idx = 0
    for vert in ordering:
        vert_to_index[vert] = idx
        idx += 1
    
    global visited; visited = set()
    global F; F = graph()
    global first; first = [0] * G.num_vertices()
    global last; last = [0] * G.num_vertices()
    global root; root = [0] * G.num_vertices()
    global Fcomp; Fcomp = [0] * G.num_vertices()
    global time; time = 0
    global fcomp; fcomp = 0
    global parent; parent = {}
    
    for vert in ordering:
        if vert not in visited:
            root[fcomp] = vert
            fcomp += 1
            DFS_on_vert(G, vert)
    
    return (last, fcomp, Fcomp, first, vert_to_index, parent)
            
def DFS_on_vert(G, v): 
    global vert_to_index; global visited; global F; global first; global last
    global root; global Fcomp; global time; global fcomp; global parent

    stack = []
    
    # In format (vertex, is ghost copy?)
    stack.append((v, False))
    
    parent[v] = None
        
    while len(stack) > 0:
        # Now we need to remove it from the parent stack
        
        curr = stack.pop()
        x = curr[0]
        
        if x not in visited:
            visited.add(x)
            time += 1
            first[vert_to_index[x]] = time
            F.Add_Edge(parent[x], x)
            Fcomp[vert_to_index[x]] = fcomp
            stack.append((x, True))
            
            if x in G.adj_list and len(G.adj_list[x]) > 0:
                curr_adj_list = sorted(G.adj_list[x], reverse=True)

                for u in curr_adj_list:
                    if u not in visited:  
                        parent[u] = x

                        stack.append((u, False))
                    
        else: 
            if curr[1] == True:
                time += 1
                last[vert_to_index[x]] = time
            else:
                continue

In [92]:
'''read in graph from graph2.txt'''
# initalize graph
G = graph()
# read in edges
G.Read_Edges("graph2.txt")
G.Read_Vertex_Labels("graph2.txt")

order = sorted(G.vertices)
print(order)
DFS(G, order)

global first
print(first)

global last
print(last)

['a', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'r', 's', 't', 'u']
[1, 21, 3, 11, 9, 2, 13, 23, 4, 6, 14, 10, 22]
[20, 26, 8, 12, 18, 19, 16, 24, 5, 7, 15, 17, 25]


We ran this by hand, and we agree with our results.

b. Using DFS routine, implement the O(n + m) time algorithm which detects whether a graph has a
cycle or not, and if so returns the cycle. Read the directed graph in the file graph3.txt and run
your code on it. Write down what you get. (10 points)



In [135]:
### DFS is a subroutine for this

def check_for_cycle(G):
    order = G.vertices
    res = DFS(G, order)
    last = res[0]
    first = res[3]
    vert_to_index = res[4]
    
    for u in G.adj_list:
        for v in G.adj_list[u]:
            # We have edge (u,v)
            # If first[v] < first[u] <last[u] < first[v]
            if first[vert_to_index[v]] < first[vert_to_index[u]] and last[vert_to_index[u]] < last[vert_to_index[v]]:
                # Recover cycle
                # get order of vertices with v as the source
                # recover path from v to u and add (u,v)
                order = list(G.vertices)
                for i in range(len(order)):
                    if order[i] == v:
                        temp = order[0]
                        order[0] = order[i]
                        order[i] = temp
                        break;
                
                res2 = DFS(G, order)
                parent = res[5]
                vert_to_index = res2[4]
                
                curr = u
                
                cycle = []
                
                while curr != v:
                    cycle = [curr] + cycle
                    # print("Taking edge: (" + str(parent[curr]) + ", "+ str(curr) + ")")
                    curr = parent[curr]
                    # print(cycle)
                    
                cycle = [v] + cycle
                cycle = [u] + cycle
                
                
                return True, cycle
    
    return False, None

In [136]:
# initalize graph
G = graph()
# read in edges
G.Read_Edges("graph3.txt")
G.Read_Vertex_Labels("graph3.txt")

print(check_for_cycle(G))

(True, ['o', 'n', 'g', 'r', 'a', 't', 's', 'c', 'o'])


c. Implement the algorithm to find strongly connected components in a directed graph. For each of
the following graphs, return the (a) number of strongly connected components, and (b) the number
of vertices in the SCC in the one with the maximum number of vertices, and (c) how many seconds
it took to run your code. Run it on a toy example to see if it’s running correctly. For instance, on
graph2.txt, the number of SCCs are 7 and the largest one 5 vertices.
You may not be able to run on the second graph if you use the recursive implementation of DFS. The
10 points are for the stack implementation

* epinions.txt (has 130K nodes and 840K edges) (5 points)
* googlemaps.txt (has 2M nodes and 2.8M edges) (10 points)

In [57]:
'''Reverses all the edges of the graph as subroutine '''

def ReverseEdges(G):
    # initalize empty graph to be gprime
    Gprime = graph()
    for v in G.vertices:
        for u in G.adj_list[v]:
            Gprime.Add_Edge(u, v)
    return Gprime

In [58]:
''' Subroutine to get the decreasing order of last[v]'s '''
def DecOrder(last, order):
    enumerated = sorted(enumerate(last), key=lambda k: k[1], reverse=True)
    
    indexes = []
    pi = []
    for i in enumerated:
        indexes.append(i[0])
    for j in indexes:
        pi.append(order[j])
    return pi

In [72]:
import time as real_time

In [77]:
'''Returns strongly connected components of G,
in topological order of G^SCC'''
def StronConnComp(G):
    start_time = real_time.time()
    # run DFS on G to get the last for every vertex
    last = DFS(G, list(G.vertices))[0] ### CHANGE SO THAT ACCESSES LAST VALUE
    # get the decreasing order of last[v]'s
    pi = DecOrder(last, list(G.vertices))
    # obtain G_rev
    GRev = ReverseEdges(G)
    # run DFS on GRev in the order of what is connected
    res = DFS(GRev, pi)
    SCC = res[1]
    FComp = res[2]
    print("We have " + str(SCC) + " connected components")
    
    start = 0
    end = 0
    biggest = 1
    
    while end < len(FComp):
        while end < len(FComp) and FComp[start] == FComp[end]:
            end += 1
            # print("Curr window: " + str(FComp[start:end]))
        biggest = max(biggest, end-start)
        start = end
    
    print("We have " + str(biggest) + " elements in our largest SCC.")
    end_time = real_time.time()
    tot_time = end_time - start_time
    print("It took " + str(tot_time) + " seconds to get these results.")

In [138]:
'''read in graph from graph2.txt'''
# initalize graph
print("Epinions test")
G = graph()
# read in edges
G.Read_Edges("epinions.txt")
G.Read_Vertex_Labels("epinions.txt")
StronConnComp(G)

Epinions test
Error: Vertex not in Graph
We have 88361 connected components
We have 31 elements in our largest SCC.
It took 11.024072885513306 seconds to get these results.


In [81]:
'''read in graph from graph2.txt'''
# initalize graph
print("Google Maps test")
G = graph()
# read in edges
G.Read_Edges("googlemaps.txt")
G.Read_Vertex_Labels("googlemaps.txt")
StronConnComp(G)

Google Maps test
We have 2638 connected components
We have 1957027 elements in our largest SCC.
It took 56.1551148891449 seconds to get these results.


d. Implement a subroutine which takes a directed graph $G$ (not necessarily acyclic) as input and finds
as long a directed path as it can. Also implement a subroutine check_path(G,p) which takes
a graph G and a list p and returns TRUE if p is indeed a path in G: so, all entries of p are vertices,
nothing is repeated in $p$, and adjacent vertices form edges. This will let you check if the path you
return is indeed a path in the graphs or not.
In the iPython/pdf file you must first clearly and succinctly describe the algorithm you will be using
to implement the above longest path routine.

Next, read the directed graphs in the files challenge1.txt and challenge2.txt and for each
of them write (a) what is the length of the path returned by your algorithm, (b) did check_path
certify this path, and (c) how long (in seconds) did your code take to do so. Furthermore, in your code
you also have to write what your paths are as a comment (so that the graders can check by running
check_path).

If there is a team which beats every other team in both graphs, then they get a +2 points. (10 points)

In [346]:
''' takes graph G and list p and returns TRUE if p is a path in G
input: graph G, array p[1:n] where n is number of verticies in the path, and there is an edge
from all verticies {n-1, n}
'''

def check_path(G, p):
    # initalize counter to store location in array
    counter = 0
    # initalize empty set of vertices that have been "seen"
    A = set()
    # go through all vertices in the path
    for i in p:
        # make sure nothing repeated; if vertex in A, has already been seen and is duplicate
        if i in A:
            return False
        # check to see that it is a vertex in graph
        if not G.isVertex(i):
            return False
        # check that the graph contains a vertex to the right
        # unless it is the last one
        # if we are not looking at the last item in our path vertex array
        if not counter == len(p)-1:
            # check if there is an edge from our vertex to the vertex to the right of it
            # if there is not, return false
            if not G.isEdge(i, p[counter+1]):
                return False
        
        A.add(i)
        counter = counter + 1
    return True

In [340]:
def graph_no_back_edge_flipped(G, order):
    #if edge pointing to vertex already seen then delete it 
    last = DFS(G, order)[0] ### CHANGE SO THAT ACCESSES LAST VALUE

    # get the decreasing order of last[v]'s
    pi = DecOrder(last, order)

    # initalize new graph to store all edges that are not back edges 
    # REMOVE ALL BACK EDGES FROM GRAPH
    
    seen = set()

    new_graph = graph()

    for v in pi:
        seen.add(v)
        # if not already, add vertex to graph
        for u in G.adj_list[v]:
            # check if it's already a vertex in the graph
            # if it is, don't add it, or the edge, since that means it's a back edge
            # if it is not yet in the graph, it is a forward edge! 
            if u not in seen:
                new_graph.Add_Vertex(u)
                new_graph.Add_Edge(u, v)

    return new_graph.adj_list, pi


# SHOULD RETURN ADJACENCY LIST OF GRAPH WITH NO BACK EDGES     

In [187]:
import random

In [337]:
def longest_path(G):
    order = list(G.vertices)
    # print("Order: " + str(order))
    random.shuffle(order)
    # print("Shuffled: " + str(order))
    res = graph_no_back_edge_flipped(G, order)
    
    adj_map = res[0]
    pi = res[1]
    
    vert_to_index = {}
    idx = 0
    for vert in pi:
        vert_to_index[vert] = idx
        idx += 1
        
    s = pi[0]
    
    # base case -- all paths are at least 1 vertex
    longest = [1] * G.num_vertices()
    parent = {}
    
    # keep track of the longest path we've seen and keep a pointer to the best vertex we've seen
    max_length = 0
    max_end = None
        
    for j in range(len(pi)):
        # Go over all in-neighbors
        if pi[j] in adj_map:
            for l in adj_map[pi[j]]:
                l = vert_to_index[l]
                # Found a new longest path -- remember its length and where it came from
                if longest[l] + 1 > longest[j]:
                    longest[j] = longest[l] + 1
                    parent[j] = l

        if j not in parent:
            parent[j] = None
        
        # Update longest path seen so far
        if max_length < longest[j]:
            max_length = longest[j]
            max_end = j
    
    # Curr is now a vertex
    curr = max_end
    path = []
    
    in_path = set()
    while curr != None:
        if curr in in_path:
            break
        in_path.add(curr)
        path = [pi[curr]] + path        
        curr = parent[curr]
        
    
    return max_length, path

In [341]:
G = graph()
# read in edges
G.Read_Edges("graph3.txt")
G.Read_Vertex_Labels("graph3.txt")

max_len = 0
best_path = None

for i in range(10):
    res = longest_path(G)
    if res[0] > max_len:
        max_len = res[0]
        best_path = res[1]
    
print(max_len)
print(best_path)

14
['m', 'n', 'g', 'r', 'a', 't', 's', 'c', 'o', 'w', 'i', 'l', 'y', 'z']


In [344]:
G = graph()
# read in edges
G.Read_Edges("challenge1.txt")
G.Read_Vertex_Labels("challenge1.txt")

max_len = 0
best_path = None

for i in range(500):
    res = longest_path(G)
    # print("Found path with length " + str(res[0]))
    if res[0] > max_len:
        print("New longest path: " + str(res[0]))
        max_len = res[0]
        best_path = res[1]
    
print(max_len)
print(best_path)

New longest path: 3359
New longest path: 3360
New longest path: 3361
New longest path: 3380
New longest path: 3391
New longest path: 3397
3397
['2667', '1684', '107', '0', '101', '180', '194', '1', '119', '100', '163', '173', '166', '198', '1902', '1006', '1039', '1040', '1063', '1056', '1029', '1059', '1004', '1003', '1075', '1012', '1113', '1025', '1080', '1001', '1095', '1300', '1179', '1114', '1155', '1026', '1028', '1017', '1024', '1078', '1074', '1048', '1110', '1047', '1076', '1079', '1086', '1032', '1014', '1009', '1066', '1018', '1099', '1081', '1020', '1102', '1108', '1235', '1015', '1021', '1007', '1067', '1019', '1060', '1100', '1036', '1129', '1082', '1104', '1044', '1158', '1215', '1192', '1120', '1115', '1190', '1148', '1227', '1159', '1010', '1000', '1134', '1127', '1090', '1122', '1237', '1030', '1043', '1045', '1183', '1051', '1042', '1052', '1011', '1041', '1072', '1037', '1035', '1013', '1109', '1070', '1092', '1054', '1128', '1049', '1112', '1184', '1083', '1091', 

In [350]:
print(check_path(G, ['2667', '1684', '107', '0', '101', '180', '194', '1', '119', '100', '163', '173', '166', '198', '1902', '1006', '1039', '1040', '1063', '1056', '1029', '1059', '1004', '1003', '1075', '1012', '1113', '1025', '1080', '1001', '1095', '1300', '1179', '1114', '1155', '1026', '1028', '1017', '1024', '1078', '1074', '1048', '1110', '1047', '1076', '1079', '1086', '1032', '1014', '1009', '1066', '1018', '1099', '1081', '1020', '1102', '1108', '1235', '1015', '1021', '1007', '1067', '1019', '1060', '1100', '1036', '1129', '1082', '1104', '1044', '1158', '1215', '1192', '1120', '1115', '1190', '1148', '1227', '1159', '1010', '1000', '1134', '1127', '1090', '1122', '1237', '1030', '1043', '1045', '1183', '1051', '1042', '1052', '1011', '1041', '1072', '1037', '1035', '1013', '1109', '1070', '1092', '1054', '1128', '1049', '1112', '1184', '1083', '1091', '1207', '1068', '1153', '1107', '1101', '1069', '1238', '1117', '1124', '1126', '1132', '1123', '1125', '1185', '1149', '1146', '1135', '1164', '1329', '1116', '1344', '1198', '1181', '1163', '1130', '1175', '1156', '1172', '1173', '1084', '1458', '1211', '1199', '1191', '1182', '1165', '1031', '1098', '1085', '1088', '1178', '1369', '1255', '1160', '1180', '1205', '1201', '1062', '1053', '1138', '1016', '1177', '1218', '1403', '1499', '1546', '1889', '1424', '1097', '1137', '1216', '1118', '1186', '1363', '1378', '1232', '1371', '1223', '1385', '1296', '1170', '1213', '1005', '1089', '1168', '1472', '1131', '1106', '1147', '1332', '1142', '1294', '1425', '1246', '1167', '1188', '1141', '1228', '1304', '1225', '1260', '1139', '1236', '1258', '1152', '1275', '1221', '1166', '1241', '1273', '1143', '1418', '129', '291', '10', '1479', '1204', '1229', '1231', '1257', '1284', '1298', '1277', '1282', '1245', '1222', '1214', '1243', '1209', '1242', '1250', '1256', '1271', '1280', '1330', '1288', '1285', '1336', '1267', '1272', '1196', '1197', '1111', '1788', '1368', '1254', '1252', '988', '1303', '1396', '1261', '1281', '1309', '1322', '1279', '1292', '1347', '1345', '1315', '1438', '1379', '1404', '1448', '1389', '1334', '1217', '1247', '1408', '1210', '1212', '1342', '1064', '1094', '1154', '1295', '1264', '1749', '1248', '1321', '1161', '1038', '1518', '1301', '1283', '1034', '2537', '1912', '136', '103', '169', '106', '231', '117', '280', '105', '148', '109', '104', '113', '121', '165', '188', '122', '128', '125', '935', '1531', '1234', '1343', '1364', '1435', '1338', '1189', '1121', '1226', '1259', '1348', '1306', '1249', '1169', '1428', '1469', '116', '1327', '1150', '1337', '1008', '1073', '1023', '1445', '1505', '1087', '1317', '1140', '1656', '1274', '1325', '1033', '1433', '1136', '1061', '1144', '1174', '1251', '1307', '1203', '1240', '1353', '2915', '2664', '2669', '1465', '1077', '1171', '1058', '1470', '1323', '1133', '1442', '1471', '1269', '1265', '1377', '1219', '1230', '1289', '1335', '1290', '1312', '1305', '1293', '1287', '1331', '1220', '1367', '1278', '1375', '1352', '1302', '1291', '1341', '1361', '1351', '1391', '1339', '1340', '1376', '1405', '1509', '1359', '1388', '1370', '1390', '1380', '1402', '1431', '1195', '1420', '1365', '1491', '1416', '1399', '1437', '1398', '1449', '1456', '1447', '1151', '1501', '1157', '1383', '1594', '1419', '1440', '1523', '1411', '1450', '1534', '1577', '1666', '1266', '1055', '1050', '1441', '1434', '1460', '1480', '1488', '1409', '1397', '1357', '1311', '1496', '1355', '1308', '1366', '1093', '1529', '1268', '1200', '1202', '1858', '1702', '1393', '1407', '1528', '1519', '1527', '1476', '1642', '1552', '1674', '1571', '1483', '1498', '1537', '1346', '2976', '2684', '2350', '1962', '1633', '1310', '1299', '1482', '123', '142', '134', '118', '161', '199', '141', '172', '13', '21', '108', '127', '135', '184', '139', '159', '197', '272', '156', '1748', '1703', '1244', '1591', '1263', '1783', '1373', '1454', '1316', '1423', '1413', '1350', '1432', '1392', '1426', '1459', '1286', '1462', '1550', '1573', '1381', '1475', '1502', '1406', '1495', '1324', '1515', '1427', '1313', '1358', '1318', '1320', '1374', '1349', '1543', '1487', '1400', '1512', '1506', '1616', '1446', '1514', '1602', '1565', '1549', '1297', '1387', '1193', '1585', '1544', '1564', '1675', '1463', '1741', '1516', '1467', '1520', '1530', '1559', '1439', '1589', '1522', '1532', '1597', '1540', '1444', '1436', '1575', '1453', '1356', '1372', '1478', '1566', '1599', '1497', '1604', '1517', '1580', '1542', '1538', '1556', '1668', '1547', '1551', '1535', '1513', '1584', '1457', '1600', '1554', '1485', '1563', '1603', '1401', '1823', '1557', '1524', '1608', '1539', '1612', '1570', '1590', '102', '175', '143', '131', '227', '225', '177', '278', '3558', '3463', '1682', '1583', '1576', '1578', '1596', '1587', '1611', '1569', '1473', '1455', '1545', '1574', '1536', '1500', '1525', '1504', '1507', '1541', '1415', '1624', '1672', '1354', '1660', '1430', '3550', '3437', '3438', '3448', '3471', '3066', '2409', '1917', '153', '3034', '2686', '2675', '2702', '111', '137', '115', '140', '144', '151', '14', '149', '20', '2', '252', '170', '124', '157', '155', '3542', '3488', '2707', '262', '2398', '1920', '1718', '1617', '1605', '1619', '1598', '1610', '1609', '1620', '1553', '1187', '1360', '1410', '1328', '1421', '1494', '1555', '1511', '1567', '1588', '1593', '1687', '1615', '164', '285', '146', '313', '203', '186', '178', '206', '2798', '2738', '2554', '1918', '1925', '1929', '1938', '1941', '1926', '1869', '1781', '1412', '1429', '1464', '1526', '1645', '1606', '1695', '1763', '1625', '1766', '1740', '1677', '1679', '1692', '1422', '1730', '1239', '1644', '1572', '1659', '1621', '1417', '1899', '1635', '1628', '1592', '1647', '1655', '1716', '1414', '1872', '1680', '1636', '1706', '1489', '1503', '1649', '1732', '1451', '1582', '1382', '1508', '1638', '1742', '1595', '1661', '1751', '1786', '1704', '1794', '1103', '1725', '1634', '1686', '1630', '1658', '1818', '1761', '1162', '1731', '741', '686', '3108', '2666', '1806', '1825', '1676', '1713', '964', '1667', '1699', '1681', '1727', '1650', '1022', '3277', '2225', '1914', '1919', '1921', '1951', '1954', '1916', '1932', '1945', '1923', '1944', '2133', '1940', '1948', '1939', '1947', '1955', '1835', '1262', '3101', '2661', '2708', '2672', '240', '222', '211', '223', '200', '185', '22', '158', '132', '242', '150', '171', '1486', '235', '273', '181', '152', '2142', '1942', '1966', '1943', '1953', '1946', '1719', '1705', '1866', '1233', '1784', '1314', '1533', '1319', '1333', '1452', '1493', '1468', '1548', '1568', '1601', '1787', '1648', '1733', '1629', '1802', '1798', '1837', '1758', '1384', '1774', '1639', '1622', '1632', '1613', '1623', '1643', '1652', '1662', '1669', '1614', '1665', '1663', '1637', '1521', '1626', '1481', '1828', '1729', '1720', '1715', '1728', '1764', '3844', '3363', '2680', '2689', '2681', '2679', '2729', '264', '2003', '1952', '1970', '1959', '1985', '1971', '1963', '1993', '1510', '1778', '1882', '1878', '1671', '1743', '1829', '1744', '1664', '1057', '1607', '1896', '3681', '3485', '3475', '3501', '3000', '2559', '1979', '1983', '1984', '1986', '2030', '1989', '2005', '1997', '2037', '2019', '2007', '1964', '1960', '1975', '1930', '2035', '2023', '1936', '1934', '2091', '2070', '2126', '1922', '1915', '2107', '2047', '1987', '2038', '2031', '2036', '1977', '2211', '2117', '1994', '2028', '2032', '2010', '2002', '2026', '1981', '2029', '2062', '2054', '2039', '2052', '2042', '2057', '2065', '2040', '2020', '2046', '1752', '1683', '1700', '1827', '1653', '1714', '1689', '1707', '1685', '1722', '1768', '1119', '3595', '3473', '3374', '2013', '1972', '1927', '1928', '1843', '1845', '1688', '1105', '1002', '1208', '3928', '3633', '3440', '3495', '3449', '3456', '3461', '3559', '3462', '3452', '3039', '2719', '2701', '2797', '2716', '2662', '1670', '1678', '1726', '2744', '2762', '1957', '1931', '1935', '1958', '1980', '1950', '120', '247', '29', '16', '329', '130', '191', '417', '348', '2919', '1027', '1627', '3276', '2668', '2727', '2795', '1814', '1696', '968', '1711', '1270', '3904', '3444', '2320', '1992', '1988', '1924', '2207', '2076', '2200', '2033', '2043', '2058', '2044', '2266', '1996', '2017', '2041', '2034', '2120', '1933', '2129', '2146', '1949', '2236', '2080', '2145', '2085', '1982', '2222', '2072', '2053', '2071', '2081', '2075', '2060', '2045', '2055', '2056', '2059', '2063', '2064', '2073', '2074', '2069', '2077', '2078', '2083', '2084', '2067', '2092', '2140', '2088', '2086', '2090', '2093', '2095', '2103', '2104', '2098', '2109', '2108', '11', '350', '384', '3923', '3720', '2790', '2760', '2678', '2536', '1885', '1807', '1709', '1756', '1831', '1443', '1910', '1694', '1790', '1844', '1750', '1717', '1618', '1819', '1721', '1724', '1737', '1757', '1712', '1736', '1646', '1813', '1723', '1746', '1734', '1775', '3515', '3511', '3506', '3486', '2293', '2068', '2087', '2102', '2111', '2110', '1973', '1968', '1976', '1991', '1206', '2791', '2665', '2711', '2714', '2979', '2764', '2677', '2358', '2230', '2099', '2197', '2130', '2156', '2015', '1937', '2357', '2152', '2008', '2114', '1965', '2049', '1978', '1990', '2173', '2162', '2313', '2051', '218', '2564', '2021', '2172', '2112', '2115', '2131', '2118', '2121', '2122', '2123', '2124', '2139', '2150', '2048', '2128', '1961', '2281', '1956', '2245', '2094', '2487', '2167', '2535', '2570', '2082', '2089', '2228', '2158', '2231', '2277', '2186', '2366', '2205', '2141', '2209', '2155', '2149', '2127', '2134', '2101', '1886', '1782', '1800', '1484', '1754', '1864', '126', '260', '322', '133', '183', '2823', '2778', '2345', '2183', '2125', '2138', '2135', '2143', '1999', '1995', '1998', '2000', '2001', '2004', '2016', '2018', '2006', '2014', '2061', '2011', '2022', '1651', '1224', '2476', '2147', '2189', '2174', '2148', '2137', '2267', '2151', '2153', '2144', '2187', '2169', '2132', '2096', '2243', '2221', '1560', '2312', '1913', '2203', '2116', '2009', '2024', '2027', '2157', '2012', '2364', '2025', '2097', '2192', '2159', '2171', '2050', '2177', '2252', '2282', '2161', '2194', '2180', '2176', '2175', '2286', '2314', '2248', '2335', '2166', '2361', '2119', '2397', '2193', '1792', '1477', '1641', '1817', '1780', '1909', '3861', '3454', '2637', '1474', '1840', '1640', '3413', '2233', '2154', '1887', '2883', '2718', '2758', '2315', '2191', '2196', '1654', '1762', '2263', '2227', '2534', '2403', '2405', '2160', '2238', '2105', '2260', '2341', '2321', '2581', '2184', '2136', '2164', '2165', '2188', '2179', '2206', '2190', '2201', '2210', '2215', '2208', '2229', '2212', '2218', '2213', '2220', '2240', '2237', '2244', '2253', '2216', '2271', '2261', '2276', '1771', '1867', '1735', '1753', '1769', '1799', '1795', '1833', '1791', '1276', '610', '221', '389', '1770', '1745', '3399', '2741', '245', '23', '190', '229', '2290', '2257', '2258', '2311', '1894', '1812', '1820', '1859', '1857', '1691', '1862', '1841', '1875', '483', '1492', '478', '2503', '2497', '2181', '2371', '2342', '2283', '2198', '2163', '2223', '2232', '2241', '2224', '2199', '2246', '2185', '2289', '2214', '174', '112', '19', '138', '842', '694', '688', '3801', '2724', '2682', '2692', '2951', '2713', '1739', '1876', '1738', '606', '1673', '1490', '1631', '366', '1767', '363', '353', '1881', '1773', '1801', '915', '1897', '2514', '2066', '2642', '2182', '2242', '2272', '2310', '2226', '2296', '2322', '2235', '2254', '2247', '2239', '2327', '2259', '2280', '2264', '2302', '2250', '2249', '2268', '2279', '2319', '17', '214', '289', '3860', '3296', '2674', '2742', '2712', '2748', '2751', '2696', '2735', '2690', '2723', '2715', '2747', '2787', '2781', '2752', '2495', '2278', '2287', '2309', '2275', '2299', '2300', '2262', '2391', '2234', '2303', '1830', '3333', '2809', '2683', '2676', '2698', '2749', '2694', '2673', '2706', '2695', '2806', '2754', '2705', '2815', '2717', '2755', '2750', '2782', '2743', '2756', '2773', '2786', '2663', '2726', '2837', '2807', '2710', '2851', '2789', '2745', '2864', '2746', '2839', '2757', '2763', '2800', '2810', '237', '25', '176', '239', '212', '232', '224', '213', '236', '248', '265', '238', '271', '261', '268', '2491', '2292', '2294', '2273', '2285', '2333', '2304', '2328', '2274', '2344', '2338', '2332', '2336', '2330', '2384', '2343', '2325', '2351', '2347', '2308', '2307', '2324', '2306', '2204', '1890', '900', '589', '3684', '2822', '2831', '2832', '2841', '2836', '2734', '2808', '3121', '2805', '2983', '2933', '2759', '2590', '2323', '2326', '2331', '2329', '2339', '2334', '2348', '2340', '2352', '1194', '869', '698', '3489', '3652', '3215', '2720', '2753', '2869', '2737', '2730', '2728', '2829', '2739', '2574', '2359', '2356', '2353', '2423', '1779', '1765', '1842', '1772', '1793', '1804', '1789', '1796', '1888', '1810', '1809', '1816', '1826', '1839', '1811', '1861', '1891', '72', '26', '257', '258', '295', '9', '297', '324', '217', '64', '189', '3709', '1836', '1852', '1561', '1863', '1883', '2910', '2777', '2827', '2852', '2916', '2866', '2867', '2880', '2835', '2862', '2873', '2872', '2850', '2877', '2553', '2354', '2363', '2369', '2376', '2370', '2374', '2392', '2395', '2381', '2407', '2404', '2386', '2408', '2410', '2387', '2393', '2449', '2427', '2454', '2478', '2288', '2383', '2305', '2572', '2385', '147', '6', '317', '24', '187', '302', '204', '196', '249', '266', '254', '2966', '2761', '2944', '1579', '3724', '3483', '3499', '3532', '2382', '2456', '784', '695', '3367', '2840', '2881', '2775', '2845', '2937', '2733', '2812', '2709', '2700', '2818', '2725', '2688', '2903', '2894', '2796', '2828', '2567', '2372', '2417', '2394', '2377', '2368', '1974', '2505', '2453', '2291', '2265', '2256', '1967', '3494', '3583', '3464', '3469', '3443', '3460', '3529', '3459', '3564', '3166', '2337', '2297', '2202', '2284', '2346', '2365', '2378', '2447', '2459', '2355', '1815', '1906', '1904', '940', '3601', '3468', '3524', '3568', '3534', '3535', '2580', '2317', '2402', '2488', '2440', '2448', '2540', '2217', '2100', '2113', '2450', '2439', '2528', '2558', '2431', '2974', '2906', '2887', '2794', '2766', '2411', '2373', '2412', '2424', '2587', '2577', '2474', '2523', '2566', '2367', '2406', '2219', '871', '858', '368', '374', '370', '376', '1777', '3004', '2890', '2905', '2891', '2821', '2731', '2874', '2907', '2920', '2442', '2499', '2430', '2428', '2414', '2446', '2460', '2462', '2418', '2433', '2464', '2467', '2429', '2489', '2468', '2390', '2396', '2415', '2318', '2270', '2416', '2362', '2419', '1581', '83', '31', '304', '168', '339', '334', '318', '62', '1871', '1848', '3340', '2826', '2813', '2803', '2772', '2785', '2687', '2780', '2765', '2793', '2833', '2863', '2770', '2849', '2853', '2939', '2901', '2904', '2811', '3162', '2779', '3019', '2830', '2990', '2870', '2697', '2493', '2641', '2525', '2557', '2295', '2502', '2298', '2425', '2594', '2401', '2170', '2255', '2486', '2426', '2490', '2349', '2513', '2912', '2897', '2909', '2884', '2106', '2421', '2634', '3510', '3466', '3554', '3149', '2911', '2925', '2896', '2931', '2854', '2927', '2908', '2960', '2924', '2928', '2929', '2993', '1880', '1395', '2652', '2434', '2399', '2438', '2432', '2509', '2451', '2420', '2458', '2436', '2463', '2445', '2471', '2461', '2475', '2498', '2512', '2389', '2400', '2413', '2529', '2530', '2521', '2482', '2469', '2500', '2477', '2485', '2484', '2492', '2526', '2506', '2507', '2504', '2546', '1832', '1868', '1898', '1879', '1849', '916', '1900', '1710', '281', '251', '284', '309', '510', '354', '361', '2859', '2847', '2856', '2784', '2848', '2996', '2819', '2783', '2984', '2804', '2992', '2936', '2868', '2825', '2844', '2871', '2875', '2958', '2882', '2968', '3011', '2685', '2957', '2946', '2913', '2948', '3023', '2945', '2943', '2940', '2981', '3025', '2898', '3214', '2771', '1907', '2972', '2379', '2571', '2472', '2465', '2555', '2508', '2510', '2516', '2519', '1698', '1847', '1803', '2842', '3239', '2970', '179', '349', '434', '373', '355', '359', '396', '367', '3153', '3054', '2985', '2956', '2969', '2953', '2988', '243', '216', '267', '110', '3653', '319', '219', '771', '3754', '3507', '3555', '3593', '3458', '2843', '2640', '2583', '2494', '2435', '2538', '2585', '2620', '2626', '2633', '2636', '2639', '2627', '2645', '2648', '2647', '2659', '2657', '2658', '2660', '896', '1824', '1893', '1850', '629', '601', '3598', '325', '277', '208', '332', '3141', '3053', '2799', '2861', '3122', '3105', '2776', '3216', '3197', '3038', '2846', '3013', '2914', '3117', '1466', '2614', '2632', '3190', '2820', '2934', '2892', '2995', '3062', '2938', '2932', '2865', '2768', '2918', '3032', '2955', '2921', '2858', '2889', '2670', '2699', '2703', '2834', '3176', '3002', '3007', '3154', '2802', '2693', '2999', '2886', '2949', '2935', '2997', '3012', '3028', '2922', '3381', '2991', '3035', '2950', '3060', '2965', '3005', '3017', '2895', '3027', '3009', '3116', '2876', '3144', '2878', '3249', '2989', '2942', '3233', '2900', '3077', '2671', '2721', '2824', '3091', '2962', '2947', '3001', '2964', '3057', '3061', '3081', '3138', '2550', '2539', '2520', '2542', '2443', '2527', '2595', '2483', '3718', '3509', '3569', '3591', '3680', '3493', '3584', '3539', '3678', '3442', '3455', '3498', '3521', '3472', '3482', '3505', '3496', '3487', '3194', '3041', '3046', '2923', '3020', '3083', '3126', '3322', '3128', '3522', '3592', '3626', '3504', '3420', '3086', '3068', '1877', '906', '2998', '3714', '3387', '2994', '3029', '1808', '1776', '205', '192', '49', '241', '802', '3543', '3497', '3450', '3545', '3480', '3540', '3556', '3492', '1461', '973', '3639', '3479', '3481', '3562', '3516', '3523', '209', '678', '395', '360', '3811', '2622', '2515', '2444', '2650', '634', '414', '34', '428', '378', '369', '357', '419', '381', '424', '486', '380', '457', '397', '3352', '3058', '3188', '3049', '3070', '3021', '2973', '3051', '3075', '3016', '3082', '3106', '3120', '3089', '3109', '3212', '3072', '3118', '3111', '3076', '3113', '3132', '3115', '3124', '3185', '3102', '3145', '3140', '3069', '3088', '3208', '3048', '3178', '3211', '3170', '3291', '3093', '2879', '2959', '2767', '3314', '2982', '3008', '3283', '2599', '2543', '2473', '2547', '2597', '2511', '2501', '2479', '2603', '2568', '375', '352', '394', '391', '400', '3174', '2986', '3014', '2977', '3084', '2987', '3026', '3010', '2769', '2838', '3003', '58', '269', '4018', '3981', '3980', '3982', '3513', '3439', '3616', '3638', '3409', '3136', '3087', '2816', '3152', '3150', '3103', '3018', '3168', '3100', '3179', '3079', '1701', '1838', '929', '38', '7', '246', '308', '323', '290', '342', '314', '50', '331', '82', '56', '207', '750', '919', '898', '613', '94', '299', '300', '320', '3112', '3172', '3182', '3198', '3107', '3024', '3160', '3240', '3187', '3156', '3255', '3167', '2801', '3221', '2954', '3248', '2736', '3327', '2893', '3432', '3279', '3097', '3043', '3110', '2855', '3280', '2646', '2551', '2549', '2532', '2552', '167', '310', '28', '226', '312', '220', '3588', '3827', '3553', '3508', '3500', '3478', '3587', '3641', '3655', '3776', '3615', '3590', '3526', '3563', '3525', '3512', '3517', '3533', '3467', '3263', '3164', '3759', '3465', '3476', '3519', '3571', '3579', '3675', '3708', '337', '32', '93', '41', '230', '71', '307', '769', '708', '890', '697', '687', '703', '709', '754', '3864', '3502', '3813', '3669', '3700', '3830', '3457', '3703', '3654', '3755', '3378', '3148', '1562', '947', '897', '1846', '922', '1805', '3254', '3343', '1854', '2629', '2533', '2582', '2588', '2598', '2518', '2480', '2610', '2531', '2616', '2589', '2608', '2649', '2605', '2643', '2592', '2653', '2609', '800', '713', '3307', '3341', '2618', '2562', '4033', '3986', '4000', '3992', '4017', '4014', '3998', '3994', '3997', '347', '3832', '315', '345', '303', '30', '48', '73', '4037', '87', '2691', '2792', '3037', '3819', '3445', '3538', '3441', '3536', '3606', '3662', '3446', '3546', '3549', '3490', '3531', '3771', '3402', '3196', '3236', '2561', '2556', '2575', '2579', '2560', '2563', '2586', '2573', '2576', '2578', '2600', '2601', '2591', '2593', '2602', '2606', '2604', '2607', '2619', '2624', '2611', '2584', '3622', '3617', '3604', '3474', '3630', '3602', '3634', '3738', '3561', '3752', '3726', '3829', '3668', '3945', '3758', '3612', '3600', '3825', '3614', '3900', '3706', '3572', '3705', '3608', '3599', '3470', '3520', '3625', '3552', '3672', '3503', '3576', '3528', '3514', '3518', '1362', '3867', '340', '671', '604', '412', '387', '383', '3782', '3090', '2888', '1856', '982', '1821', '3434', '3146', '3201', '3099', '3242', '3205', '3278', '3245', '3161', '3225', '3261', '3067', '3157', '3328', '3272', '3274', '3192', '3385', '3169', '3247', '3206', '3320', '3186', '3177', '3204', '3094', '145', '12', '3270', '3267', '3224', '3252', '3207', '3223', '3191', '3203', '3210', '3173', '3030', '3080', '2978', '3104', '3036', '2963', '3042', '3237', '3092', '2967', '1747', '1895', '958', '901', '1892', '903', '738', '705', '693', '734', '411', '2612', '676', '3913', '3596', '3557', '3050', '3312', '1865', '1911', '1860', '920', '921', '4036', '3943', '3577', '3609', '3692', '3586', '3651', '3674', '3677', '3547', '2917', '3065', '3033', '3022', '3040', '3064', '3047', '3078', '3056', '3085', '3193', '3052', '3184', '3213', '3142', '3096', '3098', '2961', '3227', '3269', '3231', '3114', '3073', '3135', '3119', '3044', '3232', '3137', '3095', '3303', '3315', '3294', '3234', '3286', '2623', '2638', '2615', '2625', '2630', '2631', '2655', '4027', '4020', '4002', '4023', '3995', '3988', '3884', '3640', '3578', '3611', '3372', '2732', '2902', '2930', '3131', '3180', '3181', '3251', '3417', '3158', '3123', '3257', '3129', '3226', '3163', '3195', '3284', '3339', '3324', '3220', '3293', '3238', '3348', '3426', '3300', '3130', '718', '716', '728', '719', '696', '711', '724', '3537', '1759', '497', '388', '3308', '3398', '3266', '3405', '3159', '3295', '3246', '3244', '619', '579', '422', '1870', '517', '398', '408', '365', '513', '2704', '3403', '3287', '3260', '3209', '3299', '3258', '3262', '3332', '3288', '3235', '3397', '3256', '2926', '3338', '3425', '3376', '3427', '3401', '1096', '957', '1873', '902', '981', '975', '622', '573', '3627', '3635', '3605', '3574', '3731', '3642', '3658', '3756', '3477', '3637', '3659', '3610', '3660', '3775', '3613', '3594', '3649', '3581', '3695', '3883', '3671', '3768', '3623', '3344', '3316', '3297', '3330', '3199', '3259', '3189', '3219', '3337', '3264', '3253', '3369', '3358', '3331', '3292', '3275', '3318', '3309', '3311', '3325', '3383', '3306', '3298', '3350', '3319', '3321', '3342', '3326', '3151', '3431', '3360', '3335', '3351', '3302', '3289', '3396', '3377', '3422', '3353', '3386', '3345', '3355', '3391', '3406', '3347', '3366', '3273', '3412', '3379', '3346', '3361', '2941', '990', '3889', '3851', '3527', '3702', '3573', '3797', '3491', '3693', '3734', '3566', '3419', '3416', '952', '934', '980', '1874', '978', '925', '927', '946', '966', '960', '997', '953', '484', '420', '430', '392', '404', '409', '431', '423', '385', '507', '402', '413', '362', '371', '542', '415', '421', '372', '410', '463', '418', '432', '416', '465', '436', '438', '444', '3925', '3530', '3551', '680', '590', '461', '439', '425', '427', '464', '552', '45', '3721', '3741', '3780', '3842', '3484', '3873', '3618', '3750', '3656', '3667', '3683', '3877', '15', '3836', '3701', '3757', '3837', '3774', '3899', '3957', '3975', '3918', '3632', '3787', '3751', '3789', '3942', '99', '263', '1760', '1693', '937', '1785', '917', '596', '574', '3370', '3620', '3646', '3643', '3648', '3631', '3786', '3645', '3764', '3719', '3711', '3784', '3793', '3763', '3799', '3647', '3716', '429', '455', '3871', '3628', '3541', '3657', '3567', '3725', '3739', '3868', '3737', '3661', '3897', '3908', '3840', '3629', '3636', '3666', '3710', '3713', '3663', '3728', '3722', '3740', '3715', '3815', '3835', '3753', '3773', '3802', '3907', '3960', '3664', '3696', '3747', '3607', '3644', '3841', '3938', '3670', '3824', '3687', '3730', '3707', '3785', '3800', '3762', '3804', '3790', '3828', '3603', '3838', '3845', '3891', '3902', '3852', '3886', '3948', '3548', '685', '498', '453', '437', '499', '532', '386', '3428', '3139', '2899', '3814', '3624', '3447', '3580', '3585', '3863', '3906', '1901', '637', '1853', '528', '407', '426', '471', '451', '514', '458', '469', '505', '509', '518', '435', '459', '516', '4004', '4013', '3989', '4011', '594', '4031', '3826', '3794', '3855', '3822', '3792', '3969', '3833', '3810', '3896', '3818', '3866', '3690', '3781', '3823', '3927', '3890', '3803', '3767', '3958', '3951', '3870', '3869', '3924', '3903', '3878', '3821', '3931', '3887', '3809', '3783', '3791', '3901', '3933', '3964', '3956', '3761', '3915', '3914', '3967', '614', '492', '445', '473', '479', '456', '440', '493', '462', '487', '446', '482', '460', '442', '475', '390', '450', '452', '382', '403', '470', '531', '405', '568', '522', '466', '504', '433', '533', '472', '567', '503', '500', '474', '481', '489', '546', '519', '506', '496', '488', '511', '512', '515', '525', '524', '520', '495', '544', '508', '534', '523', '490', '491', '502', '548', '547', '555', '553', '526', '494', '536', '538', '527', '559', '540', '480', '449', '543', '545', '537', '561', '560', '565', '327', '89', '736', '707', '714', '689', '702', '725', '732', '788', '806', '789', '3950', '3772', '3779', '3872', '3979', '3847', '3940', '3966', '3926', '3858', '3697', '3831', '3736', '3968', '3971', '635', '577', '582', '578', '583', '576', '600', '595', '615', '599', '627', '632', '2545', '817', '740', '739', '720', '730', '747', '748', '758', '745', '700', '616', '660', '584', '592', '575', '612', '603', '597', '609', '598', '605', '630', '617', '588', '618', '611', '625', '623', '636', '620', '644', '633', '666', '653', '654', '626', '669', '591', '566', '580', '936', '651', '587', '586', '645', '563', '833', '691', '746', '782', '733', '768', '712', '767', '796', '791', '818', '822', '829', '722', '737', '805', '706', '731', '690', '773', '351', '454', '798', '717', '848', '704', '753', '729', '804', '849', '723', '727', '756', '826', '766', '726', '701', '760', '781', '715', '814', '762', '799', '770', '763', '787', '755', '783', '764', '779', '765', '780', '792', '803', '710', '810', '752', '759', '828', '772', '761', '827', '797', '820', '774', '807', '819', '735', '856', '815', '812', '824', '823', '830', '778', '834', '853', '795', '813', '838', '835', '837', '3682', '3949', '3745', '3812', '3941', '3930', '3765', '554', '477', '485']))

True
